In [191]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

### Read in the required datasets

In [192]:
#### read the required data files

## counts
fnl_large_core_dm_counts_90_02 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_90_02/final_main_large_cities_core_counts_dm_1990_2002.csv')
fnl_medium_core_dm_counts_90_02 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_90_02/final_main_medium_cities_core_counts_dm_1990_2002.csv')
fnl_small_core_dm_counts_90_02 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_90_02/final_main_small_cities_core_counts_dm_1990_2002.csv')

fnl_large_core_dm_counts_03_15 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_03_15/final_main_large_cities_core_counts_dm_2003_2015.csv')
fnl_medium_core_dm_counts_03_15 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_03_15/final_main_medium_cities_core_counts_dm_2003_2015.csv')
fnl_small_core_dm_counts_03_15 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_03_15/final_main_small_cities_core_counts_dm_2003_2015.csv')

## rates
fnl_large_core_dm_rates_90_02 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_90_02/final_main_large_cities_core_rates_dm_1990_2002.csv')
fnl_medium_core_dm_rates_90_02 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_90_02/final_main_medium_cities_core_rates_dm_1990_2002.csv')
fnl_small_core_dm_rates_90_02 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_90_02/final_main_small_cities_core_rates_dm_1990_2002.csv')

fnl_large_core_dm_rates_03_15 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_03_15/final_main_large_cities_core_rates_dm_2003_2015.csv')
fnl_medium_core_dm_rates_03_15 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_03_15/final_main_medium_cities_core_rates_dm_2003_2015.csv')
fnl_small_core_dm_rates_03_15 = pd.read_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_03_15/final_main_small_cities_core_rates_dm_2003_2015.csv')

#### get median pop to be used as weights

In [193]:
def create_pop_wt_col_by_median(req_dfs):
    for df in req_dfs:
        # returns a df pop_med
        df_req = df.loc[:, ['ORI', 'fnl_population']]
        df_req_med = df_req.groupby('ORI').agg({'fnl_population':np.median})
        df_req_med.rename({'fnl_population': 'fnl_population_wt'}, axis=1, inplace=True)
        df = df.merge(df_req_med, on='ORI', how='left')
        return df

In [194]:
#### Create pop wt col for dm counts
fnl_large_core_dm_counts_90_02 = create_pop_wt_col_by_median(req_dfs=[fnl_large_core_dm_counts_90_02])
fnl_medium_core_dm_counts_90_02 = create_pop_wt_col_by_median(req_dfs=[fnl_medium_core_dm_counts_90_02])
fnl_small_core_dm_counts_90_02 = create_pop_wt_col_by_median(req_dfs=[fnl_small_core_dm_counts_90_02])

fnl_large_core_dm_counts_03_15 = create_pop_wt_col_by_median(req_dfs=[fnl_large_core_dm_counts_03_15])
fnl_medium_core_dm_counts_03_15 = create_pop_wt_col_by_median(req_dfs=[fnl_medium_core_dm_counts_03_15])
fnl_small_core_dm_counts_03_15 = create_pop_wt_col_by_median(req_dfs=[fnl_small_core_dm_counts_03_15])

In [195]:
#### Create pop wt col for dm rates
fnl_large_core_dm_rates_90_02 = create_pop_wt_col_by_median(req_dfs=[fnl_large_core_dm_rates_90_02])
fnl_medium_core_dm_rates_90_02 = create_pop_wt_col_by_median(req_dfs=[fnl_medium_core_dm_rates_90_02])
fnl_small_core_dm_rates_90_02 = create_pop_wt_col_by_median(req_dfs=[fnl_small_core_dm_rates_90_02])

fnl_large_core_dm_rates_03_15 = create_pop_wt_col_by_median(req_dfs=[fnl_large_core_dm_rates_03_15])
fnl_medium_core_dm_rates_03_15 = create_pop_wt_col_by_median(req_dfs=[fnl_medium_core_dm_rates_03_15])
fnl_small_core_dm_rates_03_15 = create_pop_wt_col_by_median(req_dfs=[fnl_small_core_dm_rates_03_15])

#### func to create lagged vars with reqd shift

In [196]:
def create_lag_vars(req_dfs, vars_to_lag, groupby_var, num_of_lags):
    for df in req_dfs:
        count=1
        for lag_num in range(num_of_lags):
            for var in vars_to_lag:
                df[f'lag{count}_{var}'] = df.groupby(f'{groupby_var}')[f'{var}'].shift(count)
            count+=1
        return df

In [197]:
#### create lags for counts
vars_lag_list_counts = ['dm_violent_crime', 'dm_total_officers', 'dm_prison_occupancy_count', 'dm_jail_occupancy_count', 'dm_drug_tot_arrests', 
               'dm_disorder_arrests_tot_index']

fnl_large_core_dm_counts_90_02 = create_lag_vars(req_dfs=[fnl_large_core_dm_counts_90_02],
                         vars_to_lag=vars_lag_list_counts,
                         groupby_var = 'ORI',
                         num_of_lags=2)
                                                     
fnl_medium_core_dm_counts_90_02 = create_lag_vars(req_dfs=[fnl_medium_core_dm_counts_90_02],
                         vars_to_lag=vars_lag_list_counts,
                         groupby_var = 'ORI',
                         num_of_lags=2)
                                                          
fnl_small_core_dm_counts_90_02 = create_lag_vars(req_dfs=[fnl_small_core_dm_counts_90_02],
                         vars_to_lag=vars_lag_list_counts,
                         groupby_var = 'ORI',
                         num_of_lags=2)
                
fnl_large_core_dm_counts_03_15 = create_lag_vars(req_dfs=[fnl_large_core_dm_counts_03_15],
                         vars_to_lag=vars_lag_list_counts,
                         groupby_var = 'ORI',
                         num_of_lags=2)
                                                     
fnl_medium_core_dm_counts_03_15 = create_lag_vars(req_dfs=[fnl_medium_core_dm_counts_03_15],
                         vars_to_lag=vars_lag_list_counts,
                         groupby_var = 'ORI',
                         num_of_lags=2)
                                                          
fnl_small_core_dm_counts_03_15 = create_lag_vars(req_dfs=[fnl_small_core_dm_counts_03_15],
                         vars_to_lag=vars_lag_list_counts,
                         groupby_var = 'ORI',
                         num_of_lags=2)

In [198]:
#### create lags for rates
vars_lag_list_rates = ['dm_violent_crime_rate', 'dm_total_officers_rate', 'dm_prison_occupancy_count_rate', 'dm_jail_occupancy_count_rate',
               'dm_drug_tot_arrests_rate', 'dm_disorder_arrests_tot_index_rate']

fnl_large_core_dm_rates_90_02 = create_lag_vars(req_dfs=[fnl_large_core_dm_rates_90_02],
                         vars_to_lag=vars_lag_list_rates,
                         groupby_var = 'ORI',
                         num_of_lags=2)
                                                     
fnl_medium_core_dm_rates_90_02 = create_lag_vars(req_dfs=[fnl_medium_core_dm_rates_90_02],
                         vars_to_lag=vars_lag_list_rates,
                         groupby_var = 'ORI',
                         num_of_lags=2)
                                                          
fnl_small_core_dm_rates_90_02 = create_lag_vars(req_dfs=[fnl_small_core_dm_rates_90_02],
                         vars_to_lag=vars_lag_list_rates,
                         groupby_var = 'ORI',
                         num_of_lags=2)
                
fnl_large_core_dm_rates_03_15 = create_lag_vars(req_dfs=[fnl_large_core_dm_rates_03_15],
                         vars_to_lag=vars_lag_list_rates,
                         groupby_var = 'ORI',
                         num_of_lags=2)
                                                     
fnl_medium_core_dm_rates_03_15 = create_lag_vars(req_dfs=[fnl_medium_core_dm_rates_03_15],
                         vars_to_lag=vars_lag_list_rates,
                         groupby_var = 'ORI',
                         num_of_lags=2)
                                                          
fnl_small_core_dm_rates_03_15 = create_lag_vars(req_dfs=[fnl_small_core_dm_rates_03_15],
                         vars_to_lag=vars_lag_list_rates,
                         groupby_var = 'ORI',
                         num_of_lags=2)

#### append all individual counts files to get top level final files

In [199]:
fnl_core_counts_dm_pop_ref_wt_lag_90_02 = fnl_large_core_dm_counts_90_02.append([fnl_medium_core_dm_counts_90_02, fnl_small_core_dm_counts_90_02], sort=False)

fnl_core_counts_dm_pop_ref_wt_lag_90_02.sort_values(['ORI', 'YEAR'], inplace=True)

fnl_core_counts_dm_pop_ref_wt_lag_90_02.to_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_pop_ref_lag/fnl_core_counts_dm_pop_ref_wt_lag_90_02.csv',
                               index=False)


fnl_core_counts_dm_pop_ref_wt_lag_03_15 = fnl_large_core_dm_counts_03_15.append([fnl_medium_core_dm_counts_03_15, fnl_small_core_dm_counts_03_15], sort=False)

fnl_core_counts_dm_pop_ref_wt_lag_03_15.sort_values(['ORI', 'YEAR'], inplace=True)

fnl_core_counts_dm_pop_ref_wt_lag_03_15.to_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_pop_ref_lag/fnl_core_counts_dm_pop_ref_wt_lag_03_15.csv',
                               index=False)


fnl_core_counts_dm_pop_ref_wt_lag_90_15 = fnl_core_counts_dm_pop_ref_wt_lag_90_02.append([fnl_core_counts_dm_pop_ref_wt_lag_03_15], sort=False)

fnl_core_counts_dm_pop_ref_wt_lag_90_15.sort_values(['ORI', 'YEAR'], inplace=True)

fnl_core_counts_dm_pop_ref_wt_lag_90_15.to_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_pop_ref_lag/fnl_core_counts_dm_pop_ref_wt_lag_90_15.csv',
                               index=False)

#### append all individual rates files to get top level final files


In [200]:
fnl_core_rates_dm_pop_ref_wt_lag_90_02 = fnl_large_core_dm_rates_90_02.append([fnl_medium_core_dm_rates_90_02, fnl_small_core_dm_rates_90_02], sort=False)

fnl_core_rates_dm_pop_ref_wt_lag_90_02.sort_values(['ORI', 'YEAR'], inplace=True)

fnl_core_rates_dm_pop_ref_wt_lag_90_02.to_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_pop_ref_lag/fnl_core_rates_dm_pop_ref_wt_lag_90_02.csv',
                               index=False)


fnl_core_rates_dm_pop_ref_wt_lag_03_15 = fnl_large_core_dm_rates_03_15.append([fnl_medium_core_dm_rates_03_15, fnl_small_core_dm_rates_03_15], sort=False)

fnl_core_rates_dm_pop_ref_wt_lag_03_15.sort_values(['ORI', 'YEAR'], inplace=True)

fnl_core_rates_dm_pop_ref_wt_lag_03_15.to_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_pop_ref_lag/fnl_core_rates_dm_pop_ref_wt_lag_03_15.csv',
                               index=False)


fnl_core_rates_dm_pop_ref_wt_lag_90_15 = fnl_core_rates_dm_pop_ref_wt_lag_90_02.append([fnl_core_rates_dm_pop_ref_wt_lag_03_15], sort=False)

fnl_core_rates_dm_pop_ref_wt_lag_90_15.sort_values(['ORI', 'YEAR'], inplace=True)

fnl_core_rates_dm_pop_ref_wt_lag_90_15.to_csv('/Users/salma/Research/us_crime_data_analysis/data/final_core_dm_pop_ref_lag/fnl_core_rates_dm_pop_ref_wt_lag_90_15.csv',
                               index=False)

#### create a func to automate weighted lagged regression - univariate for now

In [201]:
def wols_lagged_uni(req_dfs, dep_var, ind_vars, weights_var, dataset_names=False):
    for df in req_dfs:
        for var in ind_vars:
            model = sm.WLS.from_formula(f'{dep_var} ~ {var} + lag1_{var} + lag2_{var}', data=df, weights=df[f'{weights_var}'])
            results = model.fit()
            if dataset_names:
                print(dataset_names[df_count])
            print(results.summary(), '\n', '\n')


#### Rates - Perform wls with ref pop wts and lag for fnl_core_rates_dm_pop_ref_wt_lag_90_15.csv file

In [202]:
dm_dep_var_rates = 'dm_violent_crime_rate'
dm_ind_vars_rates = ['dm_total_officers_rate', 'dm_prison_occupancy_count_rate', 'dm_jail_occupancy_count_rate',
               'dm_drug_tot_arrests_rate', 'dm_disorder_arrests_tot_index_rate']

In [203]:
wols_lagged_uni(req_dfs=[fnl_core_rates_dm_pop_ref_wt_lag_90_15], 
                dep_var=dm_dep_var_rates,
               ind_vars=dm_ind_vars_rates,
               weights_var='fnl_population_wt')

                              WLS Regression Results                             
Dep. Variable:     dm_violent_crime_rate   R-squared:                       0.008
Model:                               WLS   Adj. R-squared:                  0.008
Method:                    Least Squares   F-statistic:                     230.8
Date:                   Sat, 04 Apr 2020   Prob (F-statistic):          3.70e-149
Time:                           00:10:52   Log-Likelihood:            -6.5339e+05
No. Observations:                  91031   AIC:                         1.307e+06
Df Residuals:                      91027   BIC:                         1.307e+06
Df Model:                              3                                         
Covariance Type:               nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------

In [204]:
print(list(fnl_large_core_dm_counts_90_02))

['ORI', 'AGENCY', 'YEAR', 'Govt_level', 'POP100', 'place_fips', 'STATEFP', 'CNTY', 'fnl_population', 'population_mean', 'ncrp_incarc_rep_code', 'dm_POP100', 'dm_White_count', 'dm_Black_count', 'dm_Hispanic_count', 'dm_Age1524_WhiteM', 'dm_White_Males_All', 'dm_Age1524_WhiteF', 'dm_White_Females_All', 'dm_Age1524_BlackM', 'dm_Black_Males_All', 'dm_Age1524_BlackF', 'dm_Black_Females_All', 'dm_Hispanic_Males_All', 'dm_Age1524_HispanicM', 'dm_Age1524_HispanicF', 'dm_Hispanic_Females_All', 'dm_Pct_WYM', 'dm_Pct_WYF', 'dm_total_count_county', 'dm_population', 'dm_murder', 'dm_rape', 'dm_robbery', 'dm_agg_assault', 'dm_simple_assault', 'dm_burglary', 'dm_larceny', 'dm_motor_vehicle_theft', 'dm_violent_crime', 'dm_total_crime', 'dm_property_crime', 'dm_main_crime', 'dm_murder_tot_arrests', 'dm_murder_tot_arrests_black', 'dm_murder_tot_arrests_white', 'dm_rape_tot_arrests', 'dm_rape_tot_arrests_black', 'dm_rape_tot_arrests_white', 'dm_robbery_tot_arrests', 'dm_robbery_tot_arrests_black', 'dm_ro